In [ ]:
# !pip install langchain openai milvus pymilvus python-dotenv tiktoken

In [1]:
from langchain import PromptTemplate
template = """Question: {question}

Answer: """

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

user_question = "Which NFL team won the SuperBowl in 2010?"

In [3]:
import os
from dotenv import load_dotenv
import openai
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
openai.Completion.create(
  model="text-davinci-003",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)

<OpenAIObject text_completion id=cmpl-77pkGi02MvIH8GXPL2FhnU6PUTrZi at 0x107ebd120> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is indeed a test"
    }
  ],
  "created": 1682101560,
  "id": "cmpl-77pkGi02MvIH8GXPL2FhnU6PUTrZi",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}

In [4]:
from langchain.llms import OpenAI
davinci = OpenAI(model_name="text-davinci-003")

In [5]:
from langchain import LLMChain
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The New Orleans Saints won the Super Bowl in the 2010 season.
If you are 6 ft 4 inches, you are 193.04 centimeters tall.
The 12th person on the moon was Harrison Schmitt.
A blade of grass does not have any eyes.


In [9]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]


# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [10]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What is the meaning of life?
AI: 42



User: What is the weather like today?
AI: Cloudy with a chance of memes.



User: What is your favorite movie?
AI: Terminator



User: Who is your best friend?
AI: Siri. We have spirited debates about the meaning of life.



User: What should I do today?
AI: Stop talking to chatbots on the internet and go outside.



User: What is the meaning of life?
AI: 


In [11]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length (in words) that examples should be
)

In [12]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

In [13]:
print(dynamic_prompt_template.format(query="How do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: How do birds fly?
AI: 


In [14]:
fs_llm_chain = LLMChain(
    prompt=few_shot_prompt_template,
    llm=davinci
)
fs_llm_chain.run(few_shot_prompt_template.format(query=query))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


' Life is what you make it. Make it meaningful.'

In [16]:
d_llm_chain = LLMChain(
    prompt=dynamic_prompt_template,
    llm=davinci
)
d_llm_chain.run(dynamic_prompt_template.format(query=query))

' Life is what you make of it. Make it the best you can and enjoy the journey.'

In [17]:
from langchain import ConversationChain
convo = ConversationChain(llm=davinci)

In [18]:
from langchain.chains.conversation.memory import ConversationBufferMemory
conv_buf = ConversationChain(
    llm=davinci,
    memory=ConversationBufferMemory()
)
conv_buf("Good Afternoon AI")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


{'input': 'Good Afternoon AI',
 'history': '',
 'response': ' Good afternoon! How may I help you today?'}

In [19]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
    return result

In [20]:
count_tokens(
    conv_buf, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 120 tokens


" Interesting! That's a great topic. Could you tell me more about what you hope to achieve with this integration?"

In [21]:
count_tokens(
    conv_buf,
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 204 tokens


" Hmm, that's a great question. I think one possibility could be to use a large language model to generate more accurate natural language processing results by taking advantage of external knowledge sources. Another possibility could be to use the language model to generate more detailed summaries of a document by extracting relevant knowledge from external sources."

In [22]:
count_tokens(
    conv_buf, 
    "Which external sources could be used to give context to the model?"
)

Spent a total of 251 tokens


' Some possible sources of external knowledge that could be used include databases, wikis, thesauri, ontologies, and online forums.'

In [28]:
print(conv_buf.memory.buffer)

Human: Good Afternoon AI
AI:  Good afternoon! How may I help you today?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! That's a great topic. Could you tell me more about what you hope to achieve with this integration?
Human: I just want to analyze the different possibilities. What can you think of?
AI:  Hmm, that's a great question. I think one possibility could be to use a large language model to generate more accurate natural language processing results by taking advantage of external knowledge sources. Another possibility could be to use the language model to generate more detailed summaries of a document by extracting relevant knowledge from external sources.
Human: Which external sources could be used to give context to the model?
AI:  Some possible sources of external knowledge that could be used include databases, wikis, thesauri, ontologies, and online forums.


In [31]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

convo_sum = ConversationChain(
    llm=davinci,
    memory=ConversationSummaryMemory(llm=davinci)
)

In [32]:
print(convo_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [33]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=davinci)

In [34]:
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [36]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [37]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=davinci
)

In [38]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=davinci,
    verbose=True,
    max_iterations=3
)

In [39]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
 I need to do some calculations
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [41]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=davinci, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [42]:
tools.append(llm_tool)

# reinitialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=davinci,
    verbose=True,
    max_iterations=3
)

In [43]:
zero_shot_agent("What is the capitol of Norway?")



> Entering new AgentExecutor chain...
 I need to find a way to answer this question
Action: Language Model
Action Input: "What is the capital of Norway?"
Observation: 

Oslo is the capital of Norway.
Thought: I now know the final answer
Final Answer: Oslo is the capital of Norway.

> Finished chain.


{'input': 'What is the capitol of Norway?',
 'output': 'Oslo is the capital of Norway.'}

In [44]:
zero_shot_agent("What is 6^3.1")



> Entering new AgentExecutor chain...
 I need to calculate this
Action: Calculator
Action Input: 6^3.1
Observation: Answer: 258.3859389518842
Thought: I now know the final answer
Final Answer: 258.3859389518842

> Finished chain.


{'input': 'What is 6^3.1', 'output': '258.3859389518842'}

In [47]:
# !pip install wikipedia
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="Search Wikipedia"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="Lookup a term in Wikipedia"
    )
]

In [49]:
docstore_agent = initialize_agent(
    tools, 
    llm=davinci, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)

In [52]:
docstore_agent("What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM Output: Answer: 1,800 to 7,000 ft.

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Answer: Richard Nixon.

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Answer: The Saimaa Gesture.

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Answer: Director, screenwriter, and actor.

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Answer: Arthur’s Magazine.

Question: Were Pavel Urysohn and Leonid Levin known for the same type of work?
Answer: Yes.